In [1]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint
import numpy as np
import torch
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import MDS
from plotly import tools
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from matplotlib.pyplot import figure
from sklearn.metrics import calinski_harabaz_score
from models import InferSent

In [ ]:
### encoder: Infersent need to direct to infersent folder
Information about Infersent:
universal sentence representations trained using the supervised data of the Stanford Natural Language Inference
datasets. It consistently outperform unsupervised methods like SkipThought vectors (Kiros et al., 2015) on a wide range of transfer tasks


In [2]:
# Load pre-trained model (in encoder/):
MODEL_PATH =  '/home/lucia/phd_work/InferSent/encoder/infersent1.pkl'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 1}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

In [6]:
#Set word vector path for the model:
W2V_PATH = '/home/lucia/phd_work/InferSent/dataset/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [7]:
#Build the vocabulary of word vectors (i.e keep only those needed):
model.build_vocab_k_words(K=100000)

Vocab size : 100000


## encode sentences

In [175]:
#read cleaned questions
path = '/home/lucia/phd_work/shareTask/CLpsych/'  #change to the path where you store Infersent
pos1 = pd.read_csv(  path +'/data_sample_clpsych19/user10146.posts.csv')
pos2 = pd.read_csv(  path +'/data_sample_clpsych19/user10276.posts.csv')
neg1 = pd.read_csv( path +'/data_sample_clpsych19/user-29175.posts.csv')
neg2 = pd.read_csv( path +'/data_sample_clpsych19/user-44823.posts.csv')
file1 = pd.concat([pos1, pos2], ignore_index= True)
file2 = pd.concat([neg1, neg2], ignore_index= True)
file = pd.concat([file1, file2], ignore_index= True) 
file[1:5]

,post_id,user_id,timestamp,subreddit,post_title,post_body
1,275dra,10146,1401746747,OpTicGaming,[Throwback: BO2 MLG Anaheim] WHO'S THE BEST IN...,NaN
2,2760t9,10146,1401761548,ForeverAloneDating,20 [M4F] WA - Looking for the player 2 to my p...,"What's going on everybody? My name is Waf3l, b..."
3,277upp,10146,1401816803,CoDCompetitive,Who is your favorite Pro Team?,"I know we have the flair to back it up, but I ..."
4,27cmpe,10146,1401936793,r4r,"20 [M4F] Seattle - Can you be my ""It Girl""?","What's going on everybody? My name is Waf3l, b..."


In [96]:
ps = PorterStemmer()
def processed(sent):
    words = str(sent).split()
    new_words = []
    for w in words:
        w = re.sub(r'[^\x00-\x7F]+',' ', w).lower()
        w = ps.stem(w)
        if w not in set(stopwords.words('english')):
            new_words.append(w)
    return ' '.join(new_words)

def sent_token(text):
    sentences = sent_tokenize(text)
    return sentences

def removePuncDigit(sent):
    words = str(sent).split()
    new_words = []
    for w in words:
        w = re.sub(r'[^\w\s]','',w)
        if not w.isdigit():
            new_words.append(w)
    return ' '.join(new_words)

file['post_body'] = file['post_body'].apply(lambda row: processed(row) if type(row) is str else False)
file['post_body'] = file['post_body'].apply(lambda row: sent_token(row) if type(row) is str else False)

data = file[file['post_body'] != False ]
data[1:5]

,post_id,user_id,timestamp,subreddit,post_title,post_body
2,2760t9,10146,1401761548,ForeverAloneDating,20 [M4F] WA - Looking for the player 2 to my p...,"[what' go everybody?, name waf3l, call _person..."
3,277upp,10146,1401816803,CoDCompetitive,Who is your favorite Pro Team?,"[know flair back up, wanna see fan base activ ..."
4,27cmpe,10146,1401936793,r4r,"20 [M4F] Seattle - Can you be my ""It Girl""?","[what' go everybody?, name waf3l, call _person..."
5,27fi22,10146,1402014574,CodAW,"This may have been done to death, but *my thou...","[*thi long post., apolog advanc c:* **leagu pl..."


In [112]:
# embeddingDict= {}
# for userid, post in zip(data['user_id'],data['post_body']):
#     for sent in post:
#         cleanSent = removePuncDigit(sent)
#         if len(cleanSent) >= 50:
#             #print(cleanSent)
#             embeddings = model.encode(cleanSent, tokenize=True)
#             embeddingDict[userid] = embeddings
            

/home/lucia/phd_work/InferSent/models.py:222: UserWarning:

volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



In [114]:
# for item in embeddingDict:
#     print(len(embeddingDict[item]))

55
79
78
77


In [150]:
embeddingDictPost= {}
for postid, post in zip(data['post_id'],data['post_body']):
    for sent in post:
        cleanSent = removePuncDigit(sent)
        if len(cleanSent) >= 50:
            #print(cleanSent)
            embeddings = model.encode(cleanSent, tokenize=True)
            embeddingDictPost[postid] = embeddings

/home/lucia/phd_work/InferSent/models.py:222: UserWarning:

volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



In [172]:
num_datapoints = len(embeddingDictPost)
max_num_sentence = max([len(embeddingDictPost[x]) for x in embeddingDictPost])
padded_matrix = np.zeros((num_datapoints, max_num_sentence, 4096))
j = 0
for item in embeddingDictPost:
    i = 0
    for dim in np.matrix(embeddingDictPost[item]):
        padded_matrix[j][i] = np.array(dim)
        i = i + 1
    j = j + 1
print(padded_matrix[0][87])
#    while len(embeddingDictPost[item]) < 142:
#         print('nn', len(embeddingDictPost[item]))
#        embeddingDictPost[item] = np.append(embeddingDictPost[item], padding)


[ 0.04079086 -0.08343279  0.01045423 ... -0.03926747 -0.03814263
 -0.01245085]


In [164]:
for item in embeddingDictPost:
    print(len(embeddingDictPost[item]))

88
61
57
61
93
55
138
57
86
85
79
79
68
55
85
65
56
53
57
60
54
52
84
94
62
59
62
75
91
61
70
53
76
86
92
60
66
50
51
83
66
86
75
82
66
68
74
142
103
96
73
51
79
60
78
52
228
86
57
111
67
54
84
82
93
60
67
72
78
101
115
66
51
81
57
65
51
77


In [198]:
file2  = file[file.post_body.notnull()]

In [189]:
import requests

def getSummary(text):
    r = requests.post(
        "https://api.deepai.org/api/summarization",
        data={
            'text': text,
        },
        headers={'api-key': '10da1e45-bbb0-4134-b53e-4a44a219d50d'}
    )
    return r.json()

In [199]:
file2['summary'] = file2['post_body'].apply(lambda x: getSummary(x))

/home/lucia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [200]:
file2
file2.to_csv(path + 'summary.csv')

,post_id,user_id,timestamp,subreddit,post_title,post_body,summary
0,26u0eb,10146,1401408839,dating,Am I doing something wrong?,"As the title says, I have no clue what I'm doi...",{'output': ' '}
2,2760t9,10146,1401761548,ForeverAloneDating,20 [M4F] WA - Looking for the player 2 to my p...,"What's going on everybody? My name is Waf3l, b...",{'output': 'I'm a 20 year old who is looking f...
3,277upp,10146,1401816803,CoDCompetitive,Who is your favorite Pro Team?,"I know we have the flair to back it up, but I ...",{'err': 'An error occurred processing this req...
4,27cmpe,10146,1401936793,r4r,"20 [M4F] Seattle - Can you be my ""It Girl""?","What's going on everybody? My name is Waf3l, b...",{'output': 'I'm a 20 year old who is looking f...
5,27fi22,10146,1402014574,CodAW,"This may have been done to death, but *my thou...",*This will be a long post. I apologize in adva...,{'output': 'I apologize in advance c:* ...
7,28fb94,10146,1403057482,r4r,[Meta] I messed up.,"Hey r4r, I met a girl on here and I messed up...","{'output': 'We hit it off, and after a week I ..."
9,28oave,10146,1403299579,GamerPals,20/M/PST - Looking for people to pub stomp wit...,Just looking for people to play with in my dow...,{'err': 'An error occurred processing this req...
14,2985q5,10146,1403856928,SuicideWatch,I'm done.,I'm just done with life. I'm nothing but a scr...,{'output': 'I'm nothing but a screw up. I'm re...
17,2c9893,10146,1406827508,XboxOneHelp,Game chat and controller issue [x-post from r/...,I have 2 problems: 1. Game chat won't ...,{'output': 'Game chat won't work in any games ...
20,2ggiul,10146,1410789086,DestinyTheGame,My one complaint about Destiny...,"I know I may get a lot of flack for this, but ...",{'err': 'An error occurred processing this req...


In [306]:
myList = ['err']

def findText(text, wordList):
    for item in wordList:
        if item in text:
            return True
    return False

file2['error'] = file2.apply(lambda row: True if findText(row["summary"], myList) else False, axis=1)
text = file2[file2['error'] == False]
text 

/home/lucia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,post_id,user_id,timestamp,subreddit,post_title,post_body,summary,error
0,26u0eb,10146,1401408839,dating,Am I doing something wrong?,"As the title says, I have no clue what I'm doi...",{'output': ' '},False
2,2760t9,10146,1401761548,ForeverAloneDating,20 [M4F] WA - Looking for the player 2 to my p...,"What's going on everybody? My name is Waf3l, b...",{'output': 'I'm a 20 year old who is looking f...,False
4,27cmpe,10146,1401936793,r4r,"20 [M4F] Seattle - Can you be my ""It Girl""?","What's going on everybody? My name is Waf3l, b...",{'output': 'I'm a 20 year old who is looking f...,False
5,27fi22,10146,1402014574,CodAW,"This may have been done to death, but *my thou...",*This will be a long post. I apologize in adva...,{'output': 'I apologize in advance c:* ...,False
7,28fb94,10146,1403057482,r4r,[Meta] I messed up.,"Hey r4r, I met a girl on here and I messed up...","{'output': 'We hit it off, and after a week I ...",False
14,2985q5,10146,1403856928,SuicideWatch,I'm done.,I'm just done with life. I'm nothing but a scr...,{'output': 'I'm nothing but a screw up. I'm re...,False
17,2c9893,10146,1406827508,XboxOneHelp,Game chat and controller issue [x-post from r/...,I have 2 problems: 1. Game chat won't ...,{'output': 'Game chat won't work in any games ...,False
30,303cdg,10146,1427166733,CoDCompetitive,[Fluff] My Predictions for Champs,I know you all might not care and some might e...,{'output': ' '},False
31,30cffr,10146,1427349191,CoDCompetitive,I need some help [x-post on r/codaw],Sooo....after a few months of not being able t...,{'output': ' '},False
32,30cfi8,10146,1427349233,CodAW,I need some help [x-post from r/codcompetitive],Sooo....after a few months of not being able t...,{'output': ' '},False


In [305]:
def getSumTex(df):
    sumDict = {}
    for postid, summary in zip(df['post_id'], df['summary']):
        text = summary['output']
        #print(text)
        text = processed(text)
        text = sent_token(text)
        sumDict[postid] = text
   
    return sumDict

#summaryText = getSumTex(text)

#text['summary2'] = text['summary'].apply(lambda row: getSumTex(x))
#     post = processed(i)
#     sent = sent_token(post)
    
# embeddingDictPost= {}
# for postid, post in zip(data['post_id'],data['post_body']):
#     for sent in post:
#         cleanSent = removePuncDigit(sent)
#         if len(cleanSent) >= 50:
#             #print(cleanSent)
#             embeddings = model.encode(cleanSent, tokenize=True)
#             embeddingDictPost[postid] = embeddings
            
text['summary'] = text['summary'].apply(lambda row: processed(row) )
# summaryText['summary'] = summaryText['summary'].apply(lambda row: sent_token(row) )
#summaryText['summary'] = summaryText['summary'].apply(lambda row: str(row)[1:-1])
summaryText

{'1093g0': [],
 '13ehhf': ["i'v recent start take xanax treat anxiety/pan attacks.",
  'start take xanax?'],
 '147cf0': ['practis least twenti times, present wa good, content wa good deliv veri well.',
  "instructor' respons end: thi worst way present fucked.",
  'anoth present morning, tomorrow absolut zero motiv prepar it.'],
 '14ojql': ["it' frustrat abl 'just converse' people.",
  'see peopl abl convers well effortlessli make thi feel worse.',
  "i'v tri note interest stuff say people, stuff see irl tv made much difference."],
 '15d9lc': ["ani altern melatonin may awar ani medic that' known help peopl eas way benzo withdrawal?"],
 '19ppkr': ['onli ever play first one came psp, mechan garbage, turn series.'],
 '1b47ut': ["recent i'v troubl absorb i'm reading."],
 '1gum8o': [],
 '1ksvkq': ["thing never flash firmwar phone idea it' done."],
 '1l3bwd': [],
 '1mu2li': ['stick _person_ upgrad midfield/defense?'],
 '1rc7wj': ["later he' walk along bridg night pretti sure jumps/fal come ba

In [289]:
summaryText

,post_id,summary
0,26u0eb,[[]]
1,2760t9,"[['i\'m 20 year old look ""it girl"" life., ', ""..."
2,27cmpe,"[['i\'m 20 year old look ""it girl"" life., ', ""..."
3,27fi22,[['apolog advanc c:* **leagu play** black op 2...
4,28fb94,"[['hit off, week know wa go stupidli post anot..."
5,2985q5,"[[""i'm noth screw up., "", ""i'm readi die.""]]"
6,2c9893,[['game chat work ani game (name cod).']]
7,303cdg,[[]]
8,30cffr,[[]]
9,30cfi8,[[]]
